В своем дз я использовал свою метрику: угол в двумерном пространстве между товаром и каким-то опорным вектором (на плоскости это эквивалетно обычный косинусной метрике между двумя товарами). Просто в этом случае нет необходимости считать метрику "каждый с каждым", а просто для каждого товара с опорным вектором и отсортировать. Это намного ускоряет вычисления, на моем компьютере не получалось посчитать "каждый с каждым". Минус заключается в том, что изза того что приходится сжимать пространство фичей до 2мерного теряется точность, у большого колличества товаров получаются одинаковые фичи, к сожалению я это поздно обнаружил. Но логически этот метод, на мой взгляд, правильный.
Таким образом  я сжимаю пространство до 2мерного, считаю углы, сортирую, таким образом близкие товары оказываются рядом.

In [49]:
import re
import gensim
import numpy as np
import pandas as pd
from scipy import spatial
from lightfm import LightFM
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import atan2
from scipy.sparse import load_npz
import scipy.sparse
from sklearn.decomposition import TruncatedSVD

In [50]:
Products_features = load_npz('item_features_matrix.npz')

In [51]:
I = pd.read_csv("Interactions.csv", delimiter=',')

In [52]:
I.shape

(12913475, 3)

In [53]:
print(I.product_id.max())
print(I.product_id.nunique())

509399
383771


In [54]:
#Отделяем id товаров от фичей
Products_features = Products_features.tocsc()
ids = Products_features[:,0]
Products_features = Products_features[:,1:]

In [55]:

svd = TruncatedSVD(n_components=2)
svd.fit(Products_features)
Products_features = svd.transform(Products_features)


In [56]:
Products_features

array([[ 0.06759639,  0.03306566],
       [ 0.1043013 ,  0.05107146],
       [ 0.04877673,  0.01877074],
       ...,
       [ 0.16296794,  0.43468094],
       [ 0.03304127,  0.00942511],
       [ 0.08100738, -0.00787768]])

In [57]:
t_df = pd.DataFrame(data={'id': [i[0] for i in ids[:,0].toarray().astype(int)],'x':[i for i in Products_features[:,0]], 'y':[i for i in Products_features[:,1]]})


In [58]:
t_df.head()

,id,x,y
0,335486,0.067596,0.033066
1,322530,0.104301,0.051071
2,57032,0.048777,0.018771
3,56996,0.061605,0.028936
4,37084,0.116844,0.247613


# Content-based


In [59]:
#отсортируем все по отношению к вектору (1,0) используя в качестве меры угол. Эта метрика в случае двумерного пространства идентична косинусной мере между двумя товарами. Нужно только отдельно рассмотреть пятерку товаров по краям списка ( изза свойства угла fi = fi + 2pi). Но в данном случае они (товары) очень близки к друг другу по краям списка. Такая мера в разы ускоряет вычисления
def calc_angle(point):
    return atan2(point[1],point[0])


In [60]:
#t_df['cos']= cosine_similarity(np.array([1,0]).reshape(1,2), t_df[['x','y']].values[:,:])[0]

In [61]:
t_df['angle'] = [calc_angle(point) for point in t_df[['x','y']].values[:,:]]

In [62]:
t_df.head()

,id,x,y,angle
0,335486,0.067596,0.033066,0.454941
1,322530,0.104301,0.051071,0.455336
2,57032,0.048777,0.018771,0.367361
3,56996,0.061605,0.028936,0.439113
4,37084,0.116844,0.247613,1.129897


In [63]:
t_df = t_df.sort_values(by='angle')
t_df = t_df.reset_index(drop=True)
t_df.head()

,id,x,y,angle
0,350461,0.372745,-0.277535,-0.640017
1,269201,0.372745,-0.277535,-0.640017
2,433748,0.372745,-0.277535,-0.640017
3,169194,0.372745,-0.277535,-0.640017
4,96027,0.372745,-0.277535,-0.640017


In [47]:
#save csv
with open('RadionovMaksim1.csv','w') as out:
    for idx in range(t_df.shape[0]-5):
        row =        t_df.iloc[idx] 
        cur_angle = row.angle
        for i in range(1,6):
            out.write(str(int(row.id))+' '+str(int(t_df.iloc[idx + i].id))+' '+str(t_df.iloc[idx + i].angle - cur_angle)+'\n' )
    for idx in range(t_df.shape[0]-1, t_df.shape[0]-1-5,-1):
        row =        t_df.iloc[idx] 
        cur_angle = row.angle
        for i in range(1,6):
            out.write(str(int(row.id))+' '+str(int(t_df.iloc[idx - i].id))+' '+str(t_df.iloc[idx - i].angle - cur_angle)+'\n' )

# Коллаборативная фильтрация

In [64]:
I.head()

,vid,product_id,page_type
0,0,0,PRODUCT
1,1,1,PRODUCT
2,3,3,CART
3,4,4,PURCHASE
4,5,5,PRODUCT


In [65]:
I = I[pd.to_numeric(I['vid'], errors='coerce').notnull()]
I = I[pd.to_numeric(I['product_id'], errors='coerce').notnull()]

In [66]:
print(I.product_id.nunique(), I.product_id.max())

383771 509399


In [67]:
rates = {'PRODUCT':1, 'CART':2, 'PURCHASE':3}

In [68]:
#создаем матрицу взаимодейстий (товар х пользователи)
coo = coo_matrix((np.array([rates[i] for i in I.page_type.values]),(I.product_id.values, I.vid.values)))
#I2 = coo_matrix((I.product_id.max(), I.vid.max()), dtype=np.int8)

In [70]:
#понижаем ее размерность
svd = TruncatedSVD(n_components=2)
coo2 =svd.fit_transform(coo)


In [72]:
p_df = pd.DataFrame(data={'product_id':range(len(coo2[:,0])),'x':[i for i in coo2[:,0]], 'y':[i for i in coo2[:,1]]})

In [73]:
p_df.head()
p_df =p_df[p_df.product_id.isin(I.product_id)]

In [74]:
#дата фрейм - товаром и их двумерных фичей
p_df.head()

,product_id,x,y
0,0,0.000337,4.569432e-05
1,1,0.000055,1.957575e-04
3,3,0.000002,8.788239e-05
4,4,0.000034,4.578236e-07
5,5,0.002143,7.597922e-04


In [75]:
p_df['angle'] = [calc_angle(point) for point in p_df[['x','y']].values[:,:]]

In [79]:
p_df=t_df.sort_values(by='angle')
p_df.head()

,id,x,y,angle
0,350461,0.372745,-0.277535,-0.640017
1,269201,0.372745,-0.277535,-0.640017
2,433748,0.372745,-0.277535,-0.640017
3,169194,0.372745,-0.277535,-0.640017
4,96027,0.372745,-0.277535,-0.640017


In [80]:
#save csv
with open('RadionovMaksim2.csv','w') as out:
    for idx in range(p_df.shape[0]-5):
        row =        p_df.iloc[idx] 
        cur_angle = row.angle
        for i in range(1,6):
            out.write(str(int(row.id))+' '+str(int(p_df.iloc[idx + i].id))+' '+str(p_df.iloc[idx + i].angle - cur_angle)+'\n' )
    for idx in range(p_df.shape[0]-1, p_df.shape[0]-1-5,-1):
        row =        p_df.iloc[idx] 
        cur_angle = row.angle
        for i in range(1,6):
            out.write(str(int(row.id))+' '+str(int(p_df.iloc[idx - i].id))+' '+str(p_df.iloc[idx - i].angle - cur_angle)+'\n' )

# Матричная факторизация

In [81]:
coo.shape

(509400, 975006)

In [82]:
model = LightFM(loss='warp', no_components=100, learning_rate=0.03, learning_schedule="adadelta")

In [83]:
model.fit(coo, epochs=5, num_threads=40, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [84]:
user_feature_bias, user_feature_embeddings = model.get_user_representations()
item_feature_bias, item_feature_embeddings = model.get_item_representations()

In [86]:
#use user_feature_emb (такая входная матрица)
#сжимаем 
svd = TruncatedSVD(n_components=2)
coo3 =svd.fit_transform(user_feature_embeddings)


In [91]:
m_df = pd.DataFrame(data={'product_id':range(len(coo3[:,0])),'x':[i for i in coo3[:,0]], 'y':[i for i in coo3[:,1]]})
m_df =m_df[m_df.product_id.isin(I.product_id)]


(509400, 3)
(383771, 3)


In [93]:
m_df['angle'] = [calc_angle(point) for point in m_df[['x','y']].values[:,:]]

In [95]:
m_df=t_df.sort_values(by='angle')
m_df.head()

,id,x,y,angle
0,350461,0.372745,-0.277535,-0.640017
1,269201,0.372745,-0.277535,-0.640017
2,433748,0.372745,-0.277535,-0.640017
3,169194,0.372745,-0.277535,-0.640017
4,96027,0.372745,-0.277535,-0.640017


In [96]:
with open('RadionovMaksim3.csv','w') as out:
    for idx in range(m_df.shape[0]-5):
        row =        m_df.iloc[idx] 
        cur_angle = row.angle
        for i in range(1,6):
            out.write(str(int(row.id))+' '+str(int(m_df.iloc[idx + i].id))+' '+str(m_df.iloc[idx + i].angle - cur_angle)+'\n' )
    for idx in range(m_df.shape[0]-1, m_df.shape[0]-1-5,-1):
        row =        m_df.iloc[idx] 
        cur_angle = row.angle
        for i in range(1,6):
            out.write(str(int(row.id))+' '+str(int(m_df.iloc[idx - i].id))+' '+str(m_df.iloc[idx - i].angle - cur_angle)+'\n' )